In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os

from dipy.core.gradients import gradient_table, GradientTable
from dipy.core.ndindex import ndindex
from dipy.core.sphere import faces_from_sphere_vertices, unique_edges
from dipy.data import get_sphere, HemiSphere, Sphere
from dipy.direction.peaks import peak_directions
from dipy.direction import (peaks_from_model, ProbabilisticDirectionGetter)
from dipy.io import read_bvals_bvecs
from dipy.io.image import save_nifti
from dipy.io.streamline import load_trk
from dipy.io.streamline import save_trk
from dipy.reconst.gqi import GeneralizedQSamplingModel
from dipy.reconst.peaks import reshape_peaks_for_visualization
from dipy.reconst.shm import sf_to_sh, sph_harm_lookup, smooth_pinv, sh_to_sf
from dipy.tracking import utils
from dipy.tracking.local import (ThresholdTissueClassifier, LocalTracking,ActTissueClassifier)
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask
from dipy.viz import window, actor, fvtk
from dipy.viz.colormap import line_colors
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,auto_response, recursive_response)


In [2]:
os.chdir('/space/neptune/1/users/srf29/diffusion/kanwishercon/db/vols_dti/mgh/kancon02/dti/')
wd = '/autofs/space/neptune/1/users/srf29/diffusion/kanwishercon/db/vols_dti/'

In [3]:
fimg = ("data.nii.gz")
img = nib.load(fimg)
data = img.get_data()

fbval=("bvals")
fbvec = ("/space/hemera/1/users/cmaffei/data_nancy/bvecs-x")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals,bvecs)
print(gtab.bvals.shape)

#importing binary mask
mask=("nodif_brain_mask.nii.gz")
img = nib.load(mask)
mask = img.get_data()
print("Mask Shape: " + str(mask.shape))

(272,)
Mask Shape: (140, 140, 78)


In [4]:
sphere642 = get_sphere('symmetric642')
sphere642.vertices = sphere642.vertices.astype('float')
sphere642.edges = sphere642.edges.astype('uint16')
sphere642.faces = sphere642.faces.astype('uint16')
sphere642.phi = sphere642.phi.astype('float')
sphere764 = get_sphere('symmetric724')
# sphere = HemiSphere.from_sphere(sphere=sphere642, tol=1e-01)
# sphere.vertices.shape

## Computing Matrix A

In [5]:
l_values = np.sqrt(gtab.bvals * 0.01506)
tmp=np.tile(l_values, (3,1))
gradsT = gtab.bvecs.T
b_vector = gradsT * tmp
b_vector = b_vector.T
gqi_vector = np.real(np.sinc(np.dot(b_vector, sphere642.vertices.T)* 1.6/np.pi))

### Computing odf for one voxel

In [ ]:
vox = data[52, 44, 60, :]
odf = np.dot(vox, gqi_vector)
odf = odf - (np.abs(odf).min())
odf = odf / (np.abs(odf).max())
# plotting values
plt.plot(odf)
plt.title('voxel_plot')
plt.show()

direction, pk, indices = peak_directions(odf, sphere642, relative_peak_threshold=.5, min_separation_angle=25)
pk
print(pk)

## Computing ODF for all volumes

In [6]:
odfs = np.dot(data, gqi_vector)
print(odfs.shape)

(140, 140, 78, 642)


#### Checking that I obtain exactly the same thing when computing odf in dipy

In [ ]:
#Compute brain mask for following steps
from dipy.segment.mask import median_otsu
maskdata, mask = median_otsu(data, 3, 1, False,
                             vol_idx=range(10, 50), dilate=2)
save_nifti('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/dsi_studio/linebyline/mask.nii.gz', mask.astype("uint8"),
               img.affine)



In [ ]:
gqmodel = GeneralizedQSamplingModel(gtab, 'standard', sampling_length=1)

gqpeaks = peaks_from_model(model=gqmodel, data=data,
                           sphere=sphere642, mask=mask,
                           relative_peak_threshold=.5,
                           min_separation_angle=25,
                           return_odf=True,
                           normalize_peaks=True)

## ODF Normalization

In [7]:
#min/max normalization
ijk = np.ascontiguousarray(np.array(np.nonzero(mask)).T)
shape = data.shape[:-1]
odfs_norm = np.zeros ((shape + (len(sphere642.vertices),)))

for (k, center) in enumerate(ijk):
    m = odfs[tuple(center.astype(np.int))].copy()
    m = m - (np.abs(m).min())
    m = m / (np.abs(m).max())
    odfs_norm[tuple(center.astype(np.int))] = m
    

/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in divide


In [ ]:
nib.save(nib.Nifti1Image(odfs_norm.astype(np.float32),
                                  img.affine, img.header), 'odfs/gqi_odfs_l14_dir642.nii.gz')



### Peaks Extraction

In [ ]:
#Extract peaks values and directions
#set maximum number of peaks relatove peak threshold and min separation angle
npeaks = 3
relative_peak_threshold=.5
min_separation_angle=25
shape=mask.shape    
peak_dirs = np.zeros ((shape + (npeaks, 3)))
peak_values = np.zeros ((shape + (npeaks,)))
peak_indices = np.zeros((shape + (npeaks,)), dtype='int')

odfs_norm =odfs_norm.astype(float)

for idx in ndindex(shape):
    if not mask[idx]:
        continue
    direction, pk, indices = peak_directions(odfs_norm[idx], sphere642, relative_peak_threshold, min_separation_angle)
    n = min(npeaks, pk.shape[0])
    
    peak_dirs[idx][:n] = direction[:n]
    peak_values[idx][:n] = pk[:n]
    peak_indices[idx][:n] = indices[:n]

## CSD

In [ ]:
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,recursive_response)
from dipy.reconst.dti import TensorModel, fractional_anisotropy, mean_diffusivity

#try recursive response
tenmodel = TensorModel(gtab)
tenfit = tenmodel.fit(data, mask)

FA = fractional_anisotropy(tenfit.evals)
MD = mean_diffusivity(tenfit.evals)
wm_mask = (np.logical_or(FA >= 0.4, (np.logical_and(FA >= 0.15, MD >= 0.0011))))

response = recursive_response(gtab, data, mask=wm_mask, sh_order=8,
                              peak_thr=0.01, init_fa=0.08,
                              init_trace=0.0021, iter=8, convergence=0.001,
                              parallel=True)

csd_model = ConstrainedSphericalDeconvModel(gtab, response)
csd_peaks = peaks_from_model(model=csd_model,
                             data=data,
                             sphere=sphere642,
                             relative_peak_threshold=.5,
                             min_separation_angle=25, mask=mask,
                             normalize_peaks=True,
                             return_odf=True)

### Visualization of the Normalized ODFs and peaks

In [ ]:

interactive = True
r = window.Renderer()

gqi_odfs_actor = actor.odf_slicer(odfs_norm, sphere=sphere642, scale=0.6, norm=True, colormap= 'jet')
# gqi_odfs_actor_new = actor.odf_slicer(odfs_gqi_new, sphere=sphere642, scale=0.6, norm=True, colormap= 'jet')
gqi_peaks_actor = actor.peak_slicer(peak_dirs, peak_values, colors=(1,0,0))
# csd_odfs_actor = actor.odf_slicer(odfs_csd, sphere=sphere642, scale=0.6, norm=False, colormap='jet')
# csd_peaks_actor = actor.peak_slicer(csd_peaks.peak_dirs, csd_peaks.peak_values, colors=(1,0,0))

gqi_odfs_actor.GetProperty().SetOpacity(0.4)

gqi_odfs_actor.display_extent(10, 10, 0, 30, 0, 50)
# gqi_odfs_actor_new.display_extent(0, 20, 10, 10, 0, 20)
gqi_peaks_actor.display_extent(25, 25, 0, 30, 0, 50)
# csd_odfs_actor.display_extent(60, 60, 0, 140, 0, 96)
# csd_peaks_actor.display_extent(83, 83, 69, 69, 59, 59)
# seedroi_actor = actor.contour_from_roi(slf_mask,
#                                        color=[0,1,1], opacity=0.5)

r.set_camera(position=(10, 0, 0))
# gqi_odfs_actor.display(z=0)
r.add(gqi_odfs_actor)
# r.add(gqi_odfs_actor_new)
r.add(gqi_peaks_actor)
# r.add(csd_odfs_actor)
# r.add(csd_peaks_actor)
# r.add(seedroi_actor)

# print('Saving illustration as gqi_odfs.png')
# window.record(r, out_path='odfs.png', size=(600, 600))
if interactive:
    window.show(r)

### Save to Mrtrix Basis

In [ ]:
# In both methods the  resulting ODF are flipped in Mrtrix even if starting from the 
# same data they are not. 

odf_mrtrix = sf_to_sh(odfs_norm, sphere642, sh_order=6, basis_type='mrtrix')

#save odf
nib.save(nib.Nifti1Image(odf_mrtrix.astype(np.float32),
                                  img.affine, img.header), '/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/prova_flip/gqi_odf_mrtrix.nii.gz')

odf_mrtrix = sf_to_sh(csd_peaks.odf, sphere642, sh_order=6, basis_type='mrtrix')

#save odf
nib.save(nib.Nifti1Image(odf_mrtrix.astype(np.float32),
                                  img.affine, img.header), '/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/prova_flip/csd_odf_mrtrix.nii.gz')


#some trickery to put the coefficients in the same order as MRtrix
# sh_coeff =  csd_peaks.shm_coeff
# sh_coeff_mrtrix = np.zeros(sh_coeff.shape)
# sh_coeff_mrtrix[...,0] = sh_coeff[...,0]
# sh_coeff_mrtrix[...,1:6] = sh_coeff[...,5:0:-1]
# sh_coeff_mrtrix[...,6:15] = sh_coeff[...,14:5:-1]

# nib.save(nib.Nifti1Image(sh_coeff_mrtrix.astype(np.float32), img.affine, img.header), '/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/csd_recursive_fod_mrtrix_prova.nii.gz')

In [ ]:
# loading odfs
fimg = ("../../scilpy_mrtrix_comparison/multishell/from_mgh/odfs/gqi_odfs_l14_dir642.nii.gz")
img = nib.load(fimg)
odfs_gqi = img.get_data()

fimg = ('all_shells/odfs_norm.nii.gz')
img = nib.load(fimg)
odfs_gqi_new = img.get_data()

# fimg = ("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/odf_recursive_csd.nii.gz")
# img = nib.load(fimg)
# odfs_csd = img.get_data()
# odfs_csd = sh_to_sf(odfs_csd, sphere642, sh_order=8)

### Probabilistic Tractography Test on Normalized ODF

In [ ]:
pmf = odfs_gqi.clip(min=0)
prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30, sphere=sphere642, pmf_threshold=0.5)

In [ ]:
#Define the seeds from a white matter binary mask
seed_mask=("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/structural/wm_mask_bin_res_con.nii.gz")
imgm = nib.load(seed_mask)
seed_mask = imgm.get_data()
seeds = random_seeds_from_mask(seed_mask, seeds_count=10)

#Import PVE maps
img_pve_gm = nib.load('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/structural/T12diff_pve_1_res_con.nii.gz')
img_pve_csf = nib.load('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/structural/T12diff_pve_0_res_con.nii.gz')
img_pve_wm = nib.load('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/structural/T12diff_pve_2_res_con.nii.gz')

# The background of the anatomical image should be added to the include_map
# to keep streamlines exiting the brain (e.g. through the brain stem). The ACT tissue classifier uses a trilinear 
# interpolation at the tracking position.
# background = np.ones(img_pve_gm.shape)
# background[(img_pve_gm.get_data() +
#             img_pve_wm.get_data() +
#             img_pve_csf.get_data()) > 0] = 0
                       
# include_map = img_pve_gm.get_data()
# include_map[background > 0] = 1
# exclude_map = img_pve_csf.get_data()

# act_classifier = ActTissueClassifier(include_map, exclude_map)
# streamline_generator = LocalTracking(
#         prob_dg, act_classifier, seeds, affine=np.eye(4),
#         step_size=0.75)
# streamlines = Streamlines(streamline_generator)
# #Save the trk file
# save_trk("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/prob_gqi_odf_norm_pmfthr04_npv10_angle45.trk",
#          streamlines,
#          img.affine,
#          shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

In [ ]:
from dipy.tracking.local import CmcTissueClassifier
voxel_size = np.average(img_pve_wm.get_header()['pixdim'][1:4])
step_size = 0.25

cmc_classifier = CmcTissueClassifier.from_pve(img_pve_wm.get_data(),
                                              img_pve_gm.get_data(),
                                              img_pve_csf.get_data(),
                                              step_size=step_size,
                                              average_voxel_size=voxel_size)

all_streamlines_cmc_classifier = LocalTracking(prob_dg,
                                               cmc_classifier,
                                               seeds, 
                                               affine=np.eye(4), step_size=0.75,
                                               return_all=True)

streamlines = Streamlines(all_streamlines_cmc_classifier)
save_trk("prova_gqilinescript_oldmasks_oldodfs.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

In [ ]:
# sphere181 = HemiSphere.from_sphere(get_sphere('symmetric362'))
sphere362 = get_sphere('symmetric362')
# sphere181.vertices = np.loadtxt('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/prova_flip/181dirtrackvis.txt', delimiter = ',')
# l_values = np.sqrt(gtab.bvals * 0.01506)
# tmp=np.tile(l_values, (3,1))
# gradsT = gtab.bvecs.T
# b_vector = gradsT * tmp
# b_vector = b_vector.T
gqi_vector = np.real(np.sinc(np.dot(b_vector, sphere362.vertices.T)* 1/np.pi))
odfs = np.dot(data, gqi_vector)
ijk = np.ascontiguousarray(np.array(np.nonzero(mask)).T)
shape = data.shape[:-1]
odfs_norm = np.zeros ((shape + (len(sphere362.vertices),)))

for (k, center) in enumerate(ijk):
    m = odfs[tuple(center.astype(np.int))].copy()
    m = m - (np.abs(m).min())
    m = m / (np.abs(m).max())
    odfs_norm[tuple(center.astype(np.int))] = m

In [ ]:
#visualizing
interactive = True
r = window.Renderer()

gqi_odfs_181_actor = actor.odf_slicer(odfs_norm, sphere=sphere181, scale=0.6, norm=False, colormap= 'jet')
r.add(gqi_odfs_181_actor)

if interactive:
    window.show(r)

In [ ]:
#putting directions as first dimension and saving 
odfs_prova = np.transpose(odfs_norm, (1, 2, 3, 0))
odfs_prova = np.transpose(odfs_prova, (1,2,3,0))
odfs_prova = np.transpose(odfs_prova, (1,2,3,0))
odfs_prova.shape
nib.save(nib.Nifti1Image(odfs_prova.astype(np.float32),
                                   img.affine, img.header), '/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/prova_flip/gqi_odfs_provatrackvis362.nii')


In [ ]:
sphere181.vertices = np.loadtxt('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/prova_flip/181dirtrackvis.txt', delimiter = ',')
faces_181 = faces_from_sphere_vertices(sphere181.vertices)
edges_181 = unique_edges (faces_181)
hemisphere_181 = HemiSphere(xyz=sphere181.vertices, faces= faces_181, edges=edges_181)
sphere_181 = hemisphere_181.mirror()